<a href="https://colab.research.google.com/github/Andreaierardi/Machine-Learning-Project/blob/master/project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
! git clone https://github.com/Andreaierardi/Machine-Learning-Project.git

Cloning into 'Machine-Learning-Project'...
fatal: could not read Username for 'https://github.com': No such device or address


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import random
import pandas as pd 


from sklearn.datasets import load_files
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
%ls

sample_data/


# ciao